<a href="https://colab.research.google.com/github/Hibi1001/practice/blob/main/section_3/02_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ハイパーパラメータの調整
精度向上のために、ハイパーパラメータを調整します。

## Optunaのインストール
ハイパーパラメータの最適化に使用するライブラリ、Optunaをインストールします。

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00


## データの準備
必要なライブラリの導入、データの読み込みと加工を行います。

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import lightgbm as lgb
import optuna

train_data = pd.read_csv("train-2.csv")  # 訓練データ
test_data = pd.read_csv("test-2.csv") # テストデータ

test_id = test_data["PassengerId"]  # 結果の提出時に使用

data = pd.concat([train_data, test_data], sort=False)  # テストデータ、訓練データを結合

# カテゴリデータの変換
data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
data["Embarked"].fillna(("S"), inplace=True)
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# 欠損値を埋める
data["Fare"].fillna(data["Fare"].mean(), inplace=True)
data["Age"].fillna(data["Age"].mean(), inplace=True)

# 新しい特徴量の作成
data["Family"] = data["Parch"] + data["SibSp"]

# 不要な特徴量の削除
data.drop(["Name", "PassengerId", "SibSp", "Parch", "Ticket", "Cabin"],
          axis=1, inplace=True)

# 入力と正解の作成
#data = pd.concat([train_data, test_data], sort=False)
train_data = data[:len(train_data)]
test_data = data[len(train_data):]
t = train_data["Survived"]  # 正解
x_train = train_data.drop("Survived", axis=1)  # 訓練時の入力
x_test = test_data.drop("Survived", axis=1)  # テスト時の入力

x_train.head()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
<ipython-input-3-951eee641040>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
<ipython-input-3-951eee641040>:20: FutureWarning: Downcasting behavior

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,0,22.0,7.2500,0,1
1,1,1,38.0,71.2833,1,1
2,3,1,26.0,7.9250,0,0
3,1,1,35.0,53.1000,0,1
4,3,0,35.0,8.0500,0,0


## データの分割
訓練用データを訓練用と検証用に分割します。

In [4]:
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t, test_size=0.3, stratify=t)
print(len(x_train))
print(len(x_valid))
print(len(t_train))
print(len(x_valid))


623
268
623
268


## ハイパーパラメータの最適化
ハイパーパラメータ最適化のための関数を用意します。  
最適化には、Optunaというライブラリを使用します。  
https://github.com/optuna/optuna  
  
また、機械学習のアルゴリズムには決定木をベースにした「LightGBM」を使います。  
LightGBMは「勾配ブースティング」の一種で使い勝手が良く、多くのKaggle上位者に使用された実績があります。  
大量の決定木を使用し、ある決定木の予測結果から誤差の大きなデータをうまく予測できるように次の決定木を作成します。  
https://lightgbm.readthedocs.io/en/latest/


In [5]:
categorical_features = ["Embarked", "Pclass", "Sex"]

def objective(trial):
    # ハイパーパラメータの探索範囲
    params = {
        "objective": "binary",  # 二値分類
        "max_bin": trial.suggest_int("max_bin", 200, 500),  # 特徴量の最大分割数
        "learning_rate": 0.05,  # 学習率
        "num_leaves": trial.suggest_int("num_leaves", 16, 128)  # 分岐の末端の最大数
    }

    # データセットの作成
    lgb_train = lgb.Dataset(x_train, t_train, categorical_feature=categorical_features)
    lgb_val = lgb.Dataset(x_valid, t_valid, reference=lgb_train, categorical_feature=categorical_features)

    # モデルの訓練
    verbose_eval = 20  # 学習過程の表示間隔
    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val],
                      num_boost_round=500,  # 学習回数の最大値
                      callbacks=[lgb.early_stopping(stopping_rounds=10,  # 連続して10回性能が向上しなければ終了
                                                    verbose=True),
                                 lgb.log_evaluation(verbose_eval)])

    y_valid = model.predict(x_valid, num_iteration=model.best_iteration)  # 訓練済みのモデルを使用
    score = log_loss(t_valid, y_valid)  # 二値の交差エントロピー誤差
    return score

Optunaを使い、ハイパーパラメータを最適化します。


In [6]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=30)

[I 2025-01-12 17:21:12,085] A new study created in memory with name: no-name-c8c5b2ac-301a-42cb-98f5-66997ac714e7


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[I 2025-01-12 17:21:14,380] Trial 0 finished with value: 0.4352583294123291 and parameters: {'max_bin': 338, 'num_leaves': 103}. Best is trial 0 with value: 0.4352583294123291.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[40]	training's binary_logloss: 0.347955	valid_1's binary_logloss: 0.441419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[I 2025-01-12 17:21:16,042] Trial 1 finished with value: 0.4352583294123291 and parameters: {'max_bin': 388, 'num_leaves': 102}. Best is trial 0 with value: 0.4352583294123291.


...

[I 2025-01-12 17:21:33,384] Trial 27 finished with value: 0.43729234128457384 and parameters: {'max_bin': 369, 'num_leaves': 24}. Best is trial 21 with value: 0.43188770843094665.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	training's binary_logloss: 0.429708	valid_1's binary_logloss: 0.477059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[40]	training's binary_logloss: 0.34802	valid_1's binary_logloss: 0.441359
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	training's binary_logloss: 0.2973

[I 2025-01-12 17:21:33,797] Trial 28 finished with value: 0.4352583294123291 and parameters: {'max_bin': 418, 'num_leaves': 110}. Best is trial 21 with value: 0.43188770843094665.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	training's binary_logloss: 0.429707	valid_1's binary_logloss: 0.477069
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

[I 2025-01-12 17:21:35,298] Trial 29 finished with value: 0.4352583294123291 and parameters: {'max_bin': 289, 'num_leaves': 30}. Best is trial 21 with value: 0.43188770843094665.


ベストなハイパーパラメータを表示します。

In [7]:
print(study.best_params)

{'max_bin': 212, 'num_leaves': 16}


ベストなハイパーパラメータを使って予測を行います。

In [8]:
# ベストなハイパーパラメータの設定
params = {
    "objective": "binary",  # 二値分類
    "max_bin": study.best_params["max_bin"],  # 特徴量の最大分割数
    "learning_rate": 0.05,  # 学習率
    "num_leaves": study.best_params["num_leaves"]  # 分岐の末端の最大数
}

# データセットの作成
lgb_train = lgb.Dataset(x_train, t_train, categorical_feature=categorical_features)
lgb_val = lgb.Dataset(x_valid, t_valid, reference=lgb_train, categorical_feature=categorical_features)

# モデルの訓練
verbose_eval = 20  # 学習過程の表示間隔
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val],
                    num_boost_round=500,  # 学習回数の最大値
                    callbacks=[lgb.early_stopping(stopping_rounds=10,  # 連続して10回性能が向上しなければ終了
                                                verbose=True),
                                lgb.log_evaluation(verbose_eval)])

y_test = model.predict(x_test, num_iteration=model.best_iteration)  # 訓練済みのモデルを使用

[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000094 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 252
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
Training until validation scores don't improve for 10 rounds
[20]	training's binary_logloss: 0.430343	valid_1's binary_logloss: 0.476566
[40]	training's binary_logloss: 0.352972	valid_1's binary_logloss: 0.435083
[60]	training's binary_logloss: 0.309209	valid_1's binary_logloss: 0.433896
Early stopping, best iteration is:
[50]	training's binary_logloss: 0.3286	valid_1's binary_logloss: 0.431888


## 提出用のデータ
提出量データの形式を整え、CSVファイルに保存します。

In [9]:
# 結果を0か1に
y_test = (y_test > 0.5).astype(int)

# 形式を整える
survived_test = pd.Series(y_test, name="Survived")
subm_data = pd.concat([test_id, survived_test], axis=1)

# 提出用のcsvファイルを保存
subm_data.to_csv("submission_titanic_hp.csv", index=False)

subm_data

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
